In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Series of Fork-Join Networks
val model = Network("model")

In [ ]:
// Create series of fork-join stages
val delay = Delay(model, "Delay1")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
val fork = Fork(model, "Fork")
val join = Join(model, "Join", fork)
val queue3 = Queue(model, "Queue3", SchedStrategy.PS)
val queue4 = Queue(model, "Queue4", SchedStrategy.PS)
val fork2 = Fork(model, "Fork2")
val join2 = Join(model, "Join2", fork2)

val jobclass1 = ClosedClass(model, "class1", 10, delay, 0)

In [ ]:
// Service configurations
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
delay.setService(jobclass1, Exp(0.5))
queue3.setService(jobclass1, Exp(1.0))
queue4.setService(jobclass1, Exp(1.0))

In [ ]:
// Series fork-join routing
val P = model.initRoutingMatrix()

// First fork-join stage
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
// Between stages: join -> fork2
P.set(jobclass1, jobclass1, join, fork2, 1.0)
// Second fork-join stage
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, delay, 1.0)

model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This series fork-join model demonstrates:

1. **Sequential Fork-Join Stages**:
   - **Stage 1**: Fork → {Queue1, Queue2} → Join
   - **Stage 2**: Fork2 → {Queue3, Queue4} → Join2
   - Jobs must complete both stages in sequence

2. **Pipeline Architecture**:
   - Delay → Stage1 → Stage2 → Delay (cycle)
   - Each job experiences 4 parallel operations total
   - 2 synchronization points per cycle

3. **Uniform Service Configuration**:
   - All queues have rate 1.0 (identical parallel processing)
   - Delay has rate 0.5 (bottleneck between cycles)
   - PS scheduling provides fair sharing

4. **Compound Synchronization**:
   - Each job creates 2 parallel tasks in Stage 1
   - After synchronization, creates 2 more parallel tasks in Stage 2
   - Total of 4 synchronization operations per cycle

5. **Performance Characteristics**:
   - **Delay bottleneck**: Rate 0.5 limits overall throughput
   - **Parallel amplification**: Each cycle requires 4 queue operations
   - **Load balancing**: Identical rates ensure balanced utilization

6. **Closed System Properties**:
   - 10 jobs circulate through the system
   - Jobs accumulate at slower stages
   - System reaches steady-state equilibrium

The model represents:
- **Multi-stage manufacturing**: Sequential assembly with parallel operations
- **Software pipelines**: Multi-phase processing with parallel execution
- **Data processing**: Sequential transformations with parallel computation
- **Workflow systems**: Multi-step processes with parallel task execution

Key insights:
- **Throughput limitation**: Despite 4 parallel queues, system limited by single Delay
- **Resource multiplication**: Each job requires multiple parallel resources
- **Synchronization overhead**: Multiple join points create coordination delays

**Performance Pattern**: The system throughput is determined by the slowest sequential stage (Delay), while the parallel stages provide processing capacity that must be sufficient to handle the arrival rate from that bottleneck.